# Talktorial 1

# Compound data acquisition (ChEMBL)

#### Developed in the CADD seminars 2017 and 2018, AG Volkamer, Charité/FU Berlin 

Paula Junge and Svetlana Leng - adapted by Gautier Peyrat

## Aim of this talktorial

We learn how to extract data from ChEMBL:

* Find ligands which were tested on a certain target
* Filter by available bioactivity data
* Calculate pIC50 values
* Merge dataframes and draw extracted molecules

## Learning goals


### Theory

* ChEMBL database
    * ChEMBL web services
    * ChEMBL webresource client
* Compound activity measures
    * IC50
    * pIC50

### Practical
    
Goal: Get list of compounds with bioactivity data for a given target

* Connect to ChEMBL database
* Get target data (EGFR kinase)
* Bioactivity data
    * Download and filter bioactivities
    * Clean and convert
* Compound data
    * Get list of compounds
    * Prepare output data
* Output
    * Draw molecules with highest pIC50
    * Write output file


## References

* ChEMBL bioactivity database (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5210557/)
* ChEMBL web services: <i>Nucleic Acids Res.</i> (2015), <b>43</b>, 612-620 (https://academic.oup.com/nar/article/43/W1/W612/2467881) 
* ChEMBL webrescource client GitHub (https://github.com/chembl/chembl_webresource_client)
* myChEMBL webservices version 2.x (https://github.com/chembl/mychembl/blob/master/ipython_notebooks/09_myChEMBL_web_services.ipynb)
* ChEMBL web-interface (https://www.ebi.ac.uk/chembl/)
* EBI-RDF platform (https://www.ncbi.nlm.nih.gov/pubmed/24413672)
* IC50 and pIC50 (https://en.wikipedia.org/wiki/IC50)
* UniProt website (https://www.uniprot.org/)

_____________________________________________________________________________________________________________________


## Theory

### ChEMBL database

* Open large-scale bioactivity database
* **Current data content (as of 10.2018):**
    * \>1.8 million distinct compound structures
    * \>15 million activity values from 1 million assays
    * Assays are mapped to ∼12 000 targets
* **Data sources** include scientific literature, PubChem bioassays, Drugs for Neglected Diseases Initiative (DNDi), BindingDB database, ...
* ChEMBL data can be accessed via a [web-interface](https://www.ebi.ac.uk/chembl/), the [EBI-RDF platform](https://www.ncbi.nlm.nih.gov/pubmed/24413672) and the [ChEMBL web services](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4489243/#B5)
 
    
#### ChEMBL web services

* RESTful web service
* ChEMBL web service version 2.x resource schema: 

[![ChEMBL web service schema](images/chembl_webservices_schema_diagram.jpg)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4489243/figure/F2/)

*Figure 1:* 
"ChEMBL web service schema diagram. The oval shapes represent ChEMBL web service resources and the line between two resources indicates that they share a common attribute. The arrow direction shows where the primary information about a resource type can be found. A dashed line indicates the relationship between two resources behaves differently. For example, the `Image` resource provides a graphical based representation of a `Molecule`."
Figure and description taken from: [<i>Nucleic Acids Res.</i> (2015), <b>43</b>, 612-620](https://academic.oup.com/nar/article/43/W1/W612/2467881).


#### ChEMBL webresource client

* Python client library for accessing ChEMBL data
* Handles interaction with the HTTPS protocol
* Lazy evaluation of results -> reduced number of network requests

### Compound activity measures

#### IC<sub>50</sub> 

* [Half maximal inhibitory concentration](https://en.wikipedia.org/wiki/IC50)
* Indicates how much of a particular drug or other substance is needed to inhibit a given biological process by half

<img src="https://upload.wikimedia.org/wikipedia/commons/8/81/Example_IC50_curve_demonstrating_visually_how_IC50_is_derived.png" width="450" align="center" >

*Figure 2:* Visual demonstration of how to derive an IC<sub>50</sub> value: Observe binding (or function) on vertical axis and concentration on horizontal axis; then identify max and min inhibition; IC<sub>50</sub> is the concentration at which the curve passes through the 50% inhibition level.

#### pIC<sub>50</sub>

* To facilitate the comparison of IC<sub>50</sub> values, we define pIC<sub>50</sub> values on a logarithmic scale, such that <br />
    $ pIC_{50} = -log_{10}(IC_{50}) $ where $ IC_{50}$ is specified in units of M.
* Higher pIC<sub>50</sub> values indicate exponentially greater potency of the drug
* pIC<sub>50</sub> is given in terms of molar concentration (mol/L or M) <br />
    * IC<sub>50</sub> should be specified in M to convert to pIC<sub>50</sub>  
    * For nM: $pIC_{50} = -log_{10}(IC_{50}*10^{-9})= 9-log_{10}(IC_{50}) $
    
Besides, IC<sub>50</sub> and pIC<sub>50</sub>, other bioactivity measures are used, such as the inhibition constant [Ki](https://www.ucl.ac.uk/~ucbcdab/enzass/inhibition.htm) and the half maximal effective concentration  [EC<sub>50</sub>](https://en.wikipedia.org/wiki/EC50).

## Practical

In the following, we want to download all molecules that have been tested against our target of interest, the EGFR kinase.

### Connect to ChEMBL database

First, the ChEMBL webresource client as well as other python libraries are imported.

In [ ]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import math
from rdkit.Chem import PandasTools
import os
import pickle
import sys
from nbautoeval import run_yaml_quiz

sys.path.insert(1, f'../corrections/exercices')

from T1_ChEMBL import exo_convert_nM
from T1_ChEMBL import exo_calc_pIC50

Create resource objects for API access.

In [ ]:
targets = new_client.target
compounds = new_client.molecule
bioactivities = new_client.activity

## Target data

* Get UniProt-ID (http://www.uniprot.org/uniprot/P00533) of the target of interest (EGFR kinase) from UniProt website (https://www.uniprot.org/)
* Use UniProt-ID to get target information
* Select a different UniProt-ID if you are interested into another target

In [ ]:
# define uniprot_id as "P00533"
uniprot_id =
# Get target information from ChEMBL but restrict to specified values only
target_P00533 = targets.get(target_components__accession=uniprot_id) \
                       .only('target_chembl_id', 'organism', 'pref_name', 'target_type')
print(type(target_P00533))

The results of the query are stored in the variable `target_P00533`, a `QuerySet`, i.e. the results are **NOT fetched from ChEMBL until we ask for it** (here using `pandas.DataFrame.from_records`).

More information about the `QuerySet` datatype:

    QuerySets are lazy – the act of creating a QuerySet does not involve any database activity. You can stack filters together all day long, and Django will actually not run the query until the QuerySet is evaluated.

Source: [querysets-are-lazy](https://docs.djangoproject.com/en/3.0/topics/db/queries/#querysets-are-lazy)

We have defined the `QuerySet`. Now let's fetch the results:

In [ ]:
target_P00533 = pd.DataFrame.from_records(target_P00533)
target_P00533

### After checking the entries, we select the first entry as our target of interest
`CHEMBL203`: It is a single protein and represents the human Epidermal growth factor receptor (EGFR, also named erbB1) 

In [ ]:
# select the first row of the dataframe "target_P00533"
target = 
# Check your selection by displaying the content of "target"

Save selected ChEMBL-ID.

In [ ]:
chembl_id = target['target_chembl_id']
chembl_id

### Bioactivity data

Now, we want to query bioactivity data for the target of interest.

#### Download and filter bioactivities for the target

In this step, we download and filter the bioactivity data and only consider:

* human proteins
* bioactivity type IC50
* exact measurements (relation `'='`)    
* binding data (assay type `'B'`)

In [ ]:
# Replace ### by correct arguments according to instructions given above,
# then execute the correct query:
bioact = bioactivities.filter(target_chembl_id = ### ) \
                      .filter(type = ### ) \
                      .filter(relation = ### ) \
                      .filter(assay_type = ###) \
                      .only('activity_id',
                      'assay_chembl_id', 
                      'assay_description', 
                      'assay_type',
                      'molecule_chembl_id', 
                      'type', 
                      'units', 
                      'relation', 
                      'value',
                      'target_chembl_id',
                      'target_organism')
print(f" len(bioact) = {len(bioact)} \n len(bioact[0]) = {len(bioact[0])} \n \
type(bioact) = {type(bioact)} \n type(bioact[0]) = {type(bioact[0])}")

#### Clean and convert bioactivity data

Although `bioact` is a `QuerySet`, it is stored as **list of dictionaries**:

In [ ]:
bioact

In [ ]:
# Display the first element of the queryset 'bioact'
bioact

**Remark:**  
If you experience difficulties to query the ChEMBL database, we provide here a file containing the results for the query in the previous cell (11 April 2019).  
We do this using the Python package `pickle` which serializes Python objects so they can be saved to a file, and loaded in a program again later on.
(Learn more about object serialization on [DataCamp](https://www.datacamp.com/community/tutorials/pickle-python-tutorial))

You can load the "pickled" compounds by uncommenting and running the next cell.

In [ ]:
import pickle
bioact = pickle.load(open("../data/T1/EGFR_compounds_from_chembl_query_20190411.p", "rb"))

Convert `QuerySet` to `pandas` `DataFrame` (this might take several minutes).  
However, if we use the "pickled" file above, this step will be quite fast

In [ ]:
# convert the queryset to a dataframe and store it as "bioact_df" (we already used the required method)

# then display the 10 first rows by using {dataframe_name}.head(10)


What is the shape of the `DataFrame` we obtained?

In [ ]:
bioact_df.shape

Delete entries with missing values.

In [ ]:
bioact_df = bioact_df.dropna(axis=0, how = 'any')
bioact_df.shape

Delete duplicates:
Sometimes the same molecule (`molecule_chembl_id`) has been tested more than once, in this case, we only keep the first one.

In [ ]:
# Use "drop_duplicates" method of dataframe on 'molecule_chembl_id' column
# and use correct argument to keep only the first of duplicated rows.
bioact_df = # type your answer here
bioact_df.shape

We would like to only keep bioactivity data measured in **molar** units.  
The following `print` statements will help us to see what units are contained and to control what is kept after dropping some rows.

In [ ]:
print(bioact_df.units.unique())
bioact_df = bioact_df.drop(bioact_df.index[~bioact_df.units.str.contains('M')])
print(bioact_df.units.unique())
bioact_df.shape

Since we deleted some rows, but we want to iterate over the index later, we reset index to be continuous.

In [ ]:
bioact_df = bioact_df.reset_index(drop=True) 
bioact_df.head()

To allow further comparison of the IC50 values, we convert all units to nM.
First, we write a helper function, which can be applied to the whole dataframe in the next step.

In [ ]:
exo_convert_nM.example()

In [ ]:
def convert_nM(df):
    # create an empty list "Lbioact_nM"
    # for i, row in df[['units', 'value']].iterrows(): # uncomment this line to use it directly
        # convert the bioactivity in nanomolar unit,
        # be careful with all the different unit expressions
        # If the unit is "nM", the function will return directly the value without converting it.
        # append the converted bioactivity to the list
    # Create a new colum "value" in the dataframe by using the list of bioactivites
    # Create another column "units" with the correct unit symbol
    return # return the dataframe containing converted values

In [ ]:
exo_convert_nM.correction(convert_nM)

In [ ]:
bioact_df = convert_nM(bioact_df)
# Display the first rows of the dataframe
bioact_df. # which method to use here?

### Compound data

We have a `DataFrame` containing all molecules tested (with the respective measured bioactivity) against EGFR.  
Now, we want to get the molecules that are stored behind the respective ChEMBL IDs. 

#### Get list of compounds

Let's have a look at the compounds from ChEMBL we have defined bioactivity data for. First, we retrieve ChEMBL ID and structures for the compounds with desired bioactivity data.

In [ ]:
# Get all the molecule chembl ids from bioac_df in a list called 'cmpd_id_list'

# Query the chembl by using "compounds" table with this filter : filter(molecule_chembl_id__in = cmpd_id_list)
# and select only "molecule_chembl_id" and "molecule_structures" fields, store the result in "compound_list"
# variable.


Then, we convert the `QuerySet` to `pandas` `DataFrame` and delete duplicates (It might take several minutes. So consider making an `.pickle`).

In [ ]:
# create a dataframe called "compound_df" from the query "compound_list"

# drop the duplicates and keep only the first when there is several "molecule_chembl_id"



In [ ]:
print(compound_df.shape)
print(bioact_df.shape)
compound_df.head()

So far, we have multiple different molecular structure representations. We only want to keep the canonical SMILES.

In [ ]:
for i, cmpd in compound_df.iterrows():
    if compound_df.loc[i]['molecule_structures'] != None:
        compound_df.loc[i]['molecule_structures'] = cmpd['molecule_structures']['canonical_smiles']

print(compound_df.shape)

#### Prepare output data

Merge values of interest in one dataframe on ChEMBL-IDs:
* ChEMBL-IDs
* SMILES
* units
* IC50

In [ ]:
output_df = pd.merge(bioact_df[['molecule_chembl_id','units','value']], compound_df, on='molecule_chembl_id')
print(output_df.shape)
output_df.head()

For distinct column names, we rename IC50 and SMILES columns.

In [ ]:
# rename columns in output_df dataframe : 'molecule_structures' --> 'smiles' and 'value' --> 'IC50'
# your code here

# check the shape of the obtained dataframe:
output_df.shape

If we do not have a SMILES representation of a compound, we can not further use it in the following talktorials. Therefore, we delete compounds without SMILES column.

In [ ]:
output_df = output_df[~output_df['smiles'].isnull()]
print(output_df.shape)
output_df.head()

In the next cell, you see that the low IC50 values are difficult to read. Therefore, we prefer to convert the IC50 values to pIC50.

In [ ]:
output_df = output_df.reset_index(drop=True)
ic50 = output_df.IC50.astype(float) 
print(len(ic50))
print(ic50.head(10))

In [ ]:
# create a function to convert IC50 to pIC50, 
# it takes IC50 in argument and return pIC50.
# IC50 comes as a string, it must be converted to float to calculate pIC50
# pIC50=-log10(IC50 mol/l)
# for nM: -log10(IC50*10**-9)= 9-log10(IC50)
# in the output df, a column called 'pIC50' contains all pIC50.


In [ ]:
exo_calc_pIC50.example()

In [ ]:
def calc_pIC50(df):
    ### your code here
    return df

In [ ]:
exo_calc_pIC50.correction(calc_pIC50)

In [ ]:
output_df = calc_pIC50(output_df) # apply the function on the dataframe
output_df.head()

### Collected bioactivity data for EGFR

Let's have a look at our collected data set.
#### Draw molecules
In the next steps, we add a molecule column to our datafame and look at the structures of the molecules with the highest pIC50 values. 

In [ ]:
PandasTools.AddMoleculeColumnToFrame(output_df, smilesCol='smiles')

Sort molecules by pIC50.

In [ ]:
output_df.sort_values(by="pIC50", ascending=False, inplace=True)
output_df.reset_index(drop=True, inplace=True)

Show the most active molecules = molecules with the highest pIC50 values.

In [ ]:
output_df.drop("smiles", axis=1).head()

In [ ]:
len(output_df)

#### Write output file
To use the data for the following talktorials, we save the data as `csv` file.  
Note that it is advisable to drop the molecule column (only contains an image of the molecules) when saving the data.

In [ ]:
output_df.drop("ROMol", axis=1).to_csv("../data/T1/EGFR_compounds_student.csv")

## Discussion

In this tutorial, we collected all available bioactivity data for our target of interest from the ChEMBL database. We filtered the data set to only contain molecules with measured IC50 or pIC50 bioactivity values. 

Be aware that ChEMBL data originates from various sources. Compound data has been generated in different labs by different people all over the world. Therefore, we have to be cautious with the predictions we make using this dataset. It is always important to consider the source of the data and consistency of data production assays when interpreting the results and determining how much confidence we have in our predictions.

In the next tutorials we will filter our acquired data by the Lipinski rule of five and by unwanted substructures. Another important step would be to clean the data and remove duplicates. As this is not shown in any of our talktorials (yet), we would like to refer to the standardiser library ([github Francis Atkinson](https://github.com/flatkinson/standardiser)) or [MolVS](https://molvs.readthedocs.io/en/latest/) as possible tools for this task.

# Quiz

In [ ]:
run_yaml_quiz(f"../corrections/quiz/T1_ChEMBL.yaml", "theoric-quiz")

In [ ]:
run_yaml_quiz(f"../corrections/quiz/T1_ChEMBL.yaml", "code-quiz")